# IBM Applied Data Science Capstone

## Opening new russian restaurant in Chicago


## 1. Import all libs


In [2]:
import pandas as pd
import numpy as np
import requests
!conda install -c conda-forge geopy --yes 
!conda install -c conda-forge geocoder --yes

from geopy.geocoders import Nominatim 
import geocoder

!conda install --yes  beautifulsoup4


from bs4 import BeautifulSoup
!conda install -c conda-forge folium=0.5.0 --yes 
import folium 



Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          91 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.50   | 

## 2. Extract data from web (Neighborhoods)

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

source = requests.get('https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Chicago').text

soup = BeautifulSoup(source, 'html5lib')

In [4]:
commArea = []
neighborhoodList = []

In [5]:
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        neighborhoodList.append(cells[0].text.rstrip('\n'))
        commArea.append(cells[1].text.rstrip('\n'))

In [6]:
chi_df = pd.DataFrame({"Neighborhood": neighborhoodList, "Community": commArea})

In [7]:
chi_df.head(5)

,Neighborhood,Community
0,Albany Park,Albany Park
1,Altgeld Gardens,Riverdale
2,Andersonville,Edgewater
3,Archer Heights,Archer Heights
4,Armour Square,Armour Square


In [8]:
chi_df.shape

(246, 2)

## 3. Get latitude and longitute and merge with Neighborhood

In [9]:
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Chicago, Illinois'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [11]:
coords = [ get_latlng(neighborhood) for neighborhood in chi_df["Neighborhood"].tolist() ]

In [12]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])


In [13]:
chi_df['Latitude'] = df_coords['Latitude']
chi_df['Longitude'] = df_coords['Longitude']

In [14]:
chi_df

,Neighborhood,Community,Latitude,Longitude
0,Albany Park,Albany Park,41.968290,-87.723380
1,Altgeld Gardens,Riverdale,41.654410,-87.602250
2,Andersonville,Edgewater,41.980460,-87.668340
3,Archer Heights,Archer Heights,41.811540,-87.725560
4,Armour Square,Armour Square,41.834580,-87.631890
5,Ashburn,Ashburn,41.747850,-87.709950
6,Ashburn Estates,Ashburn,41.941674,-88.198809
7,Auburn Gresham,Auburn Gresham,41.743190,-87.655040
8,Avalon Park,Avalon Park,41.745070,-87.588160
9,Avondale,Avondale,41.939250,-87.711250


In [15]:
address = 'Chicago, Illinois'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Chicago, Illinois {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Chicago, Illinois 41.8755616, -87.6244212.


In [16]:
# create map of New York using latitude and longitude values
map_chicago = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(chi_df['Latitude'], chi_df['Longitude'],chi_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_chicago)  
    
map_chicago

## 4. Use foursquare to explore the russian cousine in Chicago areas 


In [20]:
CLIENT_ID = 'EI05AG1VREBECM5VMGUXL1UN0A4A5BROAR2P5S0B42UUGC5C' # your Foursquare ID
CLIENT_SECRET = 'JH351FIX5HRK5AG0S1IUDXRAAVN3CJECC3A2ABYN1G5H2OLM' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 15
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
search_query = 'Russian'
radius = 5000
lat = 41.968290000000025
long = -87.72337999999996

Your credentails:
CLIENT_ID: EI05AG1VREBECM5VMGUXL1UN0A4A5BROAR2P5S0B42UUGC5C
CLIENT_SECRET:JH351FIX5HRK5AG0S1IUDXRAAVN3CJECC3A2ABYN1G5H2OLM


In [18]:
# url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
# url
# create the API request URL
# url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&query={},&ll={},{}&radius={}&limit={}".format(
#         CLIENT_ID,
#         CLIENT_SECRET,
#         VERSION,
#         search_query,
#         lat,
#         long,
#         radius,
#         LIMIT)
    
# results = requests.get(url).json()["response"]['groups'][0]['items']



In [19]:
# results

# for venue in results:
#         venues.append((
#             neighborhood,
#             lat, 
#             long, 
#             venue['venue']['name'], 
#             venue['venue']['location']['lat'], 
#             venue['venue']['location']['lng'],  
#             venue['venue']['categories'][0]['name']))


NameError: name 'venues' is not defined

In [114]:
venues

[('Albany Park',
  41.968290000000025,
  -87.72337999999996,
  'Jibek Jolu',
  41.97310738566933,
  -87.6914670054359,
  'Russian Restaurant'),
 ('Albany Park',
  41.968290000000025,
  -87.72337999999996,
  'Kopi Café',
  41.97861200149812,
  -87.66829842785883,
  'Café'),
 ('Albany Park',
  41.968290000000025,
  -87.72337999999996,
  'The Bad Apple',
  41.95969836444237,
  -87.6827632034987,
  'Burger Joint'),
 ('Albany Park',
  41.968290000000025,
  -87.72337999999996,
  'Bereke',
  41.96876517632379,
  -87.68289518855285,
  'Russian Restaurant'),
 ('Albany Park',
  41.968290000000025,
  -87.72337999999996,
  "Silvie's Lounge",
  41.95431871204755,
  -87.67645401243416,
  'Music Venue'),
 ('Albany Park',
  41.968290000000025,
  -87.72337999999996,
  'Jibek Jolu',
  41.97310738566933,
  -87.6914670054359,
  'Russian Restaurant'),
 ('Albany Park',
  41.968290000000025,
  -87.72337999999996,
  'Kopi Café',
  41.97861200149812,
  -87.66829842785883,
  'Café'),
 ('Albany Park',
  41.96829

In [21]:

venues = []


In [22]:

for lat, long, neighborhood in zip(chi_df['Latitude'], chi_df['Longitude'], chi_df['Neighborhood']):
    
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&query={},&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        search_query,
        lat,
        long,
        radius,
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']


# return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [23]:
venues 



[('Albany Park',
  41.968290000000025,
  -87.72337999999996,
  'Jibek Jolu',
  41.97310738566933,
  -87.6914670054359,
  'Russian Restaurant'),
 ('Albany Park',
  41.968290000000025,
  -87.72337999999996,
  'Kopi Café',
  41.97861200149812,
  -87.66829842785883,
  'Café'),
 ('Albany Park',
  41.968290000000025,
  -87.72337999999996,
  'The Bad Apple',
  41.95969836444237,
  -87.6827632034987,
  'Burger Joint'),
 ('Albany Park',
  41.968290000000025,
  -87.72337999999996,
  'Bereke',
  41.96876517632379,
  -87.68289518855285,
  'Russian Restaurant'),
 ('Albany Park',
  41.968290000000025,
  -87.72337999999996,
  "Silvie's Lounge",
  41.95431871204755,
  -87.67645401243416,
  'Music Venue'),
 ('Andersonville',
  41.98046000000005,
  -87.66833999999994,
  'Jibek Jolu',
  41.97310738566933,
  -87.6914670054359,
  'Russian Restaurant'),
 ('Andersonville',
  41.98046000000005,
  -87.66833999999994,
  'Kopi Café',
  41.97861200149812,
  -87.66829842785883,
  'Café'),
 ('Andersonville',
  41.9

In [24]:
venues_df = pd.DataFrame(venues)


In [25]:
venues_df

,0,1,2,3,4,5,6
0,Albany Park,41.968290,-87.723380,Jibek Jolu,41.973107,-87.691467,Russian Restaurant
1,Albany Park,41.968290,-87.723380,Kopi Café,41.978612,-87.668298,Café
2,Albany Park,41.968290,-87.723380,The Bad Apple,41.959698,-87.682763,Burger Joint
3,Albany Park,41.968290,-87.723380,Bereke,41.968765,-87.682895,Russian Restaurant
4,Albany Park,41.968290,-87.723380,Silvie's Lounge,41.954319,-87.676454,Music Venue
5,Andersonville,41.980460,-87.668340,Jibek Jolu,41.973107,-87.691467,Russian Restaurant
6,Andersonville,41.980460,-87.668340,Kopi Café,41.978612,-87.668298,Café
7,Andersonville,41.980460,-87.668340,The Bad Apple,41.959698,-87.682763,Burger Joint
8,Andersonville,41.980460,-87.668340,Bereke,41.968765,-87.682895,Russian Restaurant
9,Andersonville,41.980460,-87.668340,Nookies Tree Restaurant,41.942858,-87.649391,Diner


In [26]:
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']


In [27]:
venues_df["VenueName"].unique()

array(['Jibek Jolu', 'Kopi Café', 'The Bad Apple', 'Bereke',
       "Silvie's Lounge", 'Nookies Tree Restaurant', 'Schubas Tavern',
       'Sidetrack', "Manny's Cafeteria & Delicatessen",
       'Red Square Bathhouse', 'red square cafe', 'Russian Tea Time',
       'The Terrace at Trump', 'MONEYGUN', 'Mott Street',
       "Lito's Empanadas", 'Cafe Luigi'], dtype=object)

In [28]:
venues_df.groupby(["Neighborhood"]).count()


,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Albany Park,5,5,5,5,5,5
Andersonville,8,8,8,8,8,8
Armour Square,1,1,1,1,1,1
Avondale,7,7,7,7,7,7
Big Oaks,7,7,7,7,7,7
Bowmanville,8,8,8,8,8,8
Boystown,10,10,10,10,10,10
Bridgeport,1,1,1,1,1,1
Bucktown,11,11,11,11,11,11


## 5. Prepare data for clustering 

In [29]:
# one hot encoding
chi_onehot = pd.get_dummies(venues_df[['VenueName']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
chi_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [chi_onehot.columns[-1]] + list(chi_onehot.columns[:-1])
chi_onehot = chi_onehot[fixed_columns]

print(chi_onehot.shape)
chi_onehot.head()

(798, 18)


,Neighborhoods,Bereke,Cafe Luigi,Jibek Jolu,Kopi Café,Lito's Empanadas,MONEYGUN,Manny's Cafeteria & Delicatessen,Mott Street,Nookies Tree Restaurant,Red Square Bathhouse,Russian Tea Time,Schubas Tavern,Sidetrack,Silvie's Lounge,The Bad Apple,The Terrace at Trump,red square cafe
0,Albany Park,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Albany Park,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Albany Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,Albany Park,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Albany Park,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [30]:
chi_grouped = chi_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(chi_grouped.shape)
chi_grouped.head()

(119, 18)


,Neighborhoods,Bereke,Cafe Luigi,Jibek Jolu,Kopi Café,Lito's Empanadas,MONEYGUN,Manny's Cafeteria & Delicatessen,Mott Street,Nookies Tree Restaurant,Red Square Bathhouse,Russian Tea Time,Schubas Tavern,Sidetrack,Silvie's Lounge,The Bad Apple,The Terrace at Trump,red square cafe
0,Albany Park,0.200000,0.0,0.200000,0.200,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000,0.200000,0.200000,0.000000,0.000000
1,Andersonville,0.125000,0.0,0.125000,0.125,0.0,0.000000,0.000000,0.000000,0.125,0.000000,0.000000,0.125000,0.125,0.125000,0.125000,0.000000,0.000000
2,Armour Square,0.000000,0.0,0.000000,0.000,0.0,0.000000,1.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000
3,Avondale,0.142857,0.0,0.142857,0.000,0.0,0.000000,0.000000,0.000000,0.000,0.142857,0.000000,0.142857,0.000,0.142857,0.142857,0.000000,0.142857
4,Big Oaks,0.000000,0.0,0.000000,0.000,0.0,0.142857,0.142857,0.142857,0.000,0.142857,0.142857,0.000000,0.000,0.000000,0.000000,0.142857,0.142857


## 6. Cluster areas

In [31]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 4

chi_clustering = chi_grouped.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(chi_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 2, 1, 0, 1, 0, 2, 0, 0], dtype=int32)

In [32]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
chi_merged = chi_grouped.copy()

# add clustering labels
chi_merged["Cluster Labels"] = kmeans.labels_

In [33]:
chi_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
chi_merged.head()

,Neighborhood,Bereke,Cafe Luigi,Jibek Jolu,Kopi Café,Lito's Empanadas,MONEYGUN,Manny's Cafeteria & Delicatessen,Mott Street,Nookies Tree Restaurant,Red Square Bathhouse,Russian Tea Time,Schubas Tavern,Sidetrack,Silvie's Lounge,The Bad Apple,The Terrace at Trump,red square cafe,Cluster Labels
0,Albany Park,0.200000,0.0,0.200000,0.200,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000,0.200000,0.200000,0.000000,0.000000,1
1,Andersonville,0.125000,0.0,0.125000,0.125,0.0,0.000000,0.000000,0.000000,0.125,0.000000,0.000000,0.125000,0.125,0.125000,0.125000,0.000000,0.000000,1
2,Armour Square,0.000000,0.0,0.000000,0.000,0.0,0.000000,1.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,2
3,Avondale,0.142857,0.0,0.142857,0.000,0.0,0.000000,0.000000,0.000000,0.000,0.142857,0.000000,0.142857,0.000,0.142857,0.142857,0.000000,0.142857,1
4,Big Oaks,0.000000,0.0,0.000000,0.000,0.0,0.142857,0.142857,0.142857,0.000,0.142857,0.142857,0.000000,0.000,0.000000,0.000000,0.142857,0.142857,0


In [34]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
chi_merged = chi_merged.join(chi_df.set_index("Neighborhood"), on="Neighborhood")

chi_merged.head() # check the last columns!

,Neighborhood,Bereke,Cafe Luigi,Jibek Jolu,Kopi Café,Lito's Empanadas,MONEYGUN,Manny's Cafeteria & Delicatessen,Mott Street,Nookies Tree Restaurant,Red Square Bathhouse,Russian Tea Time,Schubas Tavern,Sidetrack,Silvie's Lounge,The Bad Apple,The Terrace at Trump,red square cafe,Cluster Labels,Community,Latitude,Longitude
0,Albany Park,0.200000,0.0,0.200000,0.200,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000,0.200000,0.200000,0.000000,0.000000,1,Albany Park,41.96829,-87.72338
1,Andersonville,0.125000,0.0,0.125000,0.125,0.0,0.000000,0.000000,0.000000,0.125,0.000000,0.000000,0.125000,0.125,0.125000,0.125000,0.000000,0.000000,1,Edgewater,41.98046,-87.66834
2,Armour Square,0.000000,0.0,0.000000,0.000,0.0,0.000000,1.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,2,Armour Square,41.83458,-87.63189
3,Avondale,0.142857,0.0,0.142857,0.000,0.0,0.000000,0.000000,0.000000,0.000,0.142857,0.000000,0.142857,0.000,0.142857,0.142857,0.000000,0.142857,1,Avondale,41.93925,-87.71125
4,Big Oaks,0.000000,0.0,0.000000,0.000,0.0,0.142857,0.142857,0.142857,0.000,0.142857,0.142857,0.000000,0.000,0.000000,0.000000,0.142857,0.142857,0,Norwood Park,41.88425,-87.63245


In [35]:

# sort the results by Cluster Labels
chi_merged.sort_values(["Cluster Labels"], inplace=True)
chi_merged

,Neighborhood,Bereke,Cafe Luigi,Jibek Jolu,Kopi Café,Lito's Empanadas,MONEYGUN,Manny's Cafeteria & Delicatessen,Mott Street,Nookies Tree Restaurant,Red Square Bathhouse,Russian Tea Time,Schubas Tavern,Sidetrack,Silvie's Lounge,The Bad Apple,The Terrace at Trump,red square cafe,Cluster Labels,Community,Latitude,Longitude
59,Near West Side,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.142857,0.142857,0.000000,0.142857,0.142857,0.000000,0.000000,0.000000,0.000000,0.142857,0.142857,0,Near West Side,41.873010,-87.676130
33,Heart of Chicago,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.142857,0.142857,0.000000,0.142857,0.142857,0.000000,0.000000,0.000000,0.000000,0.142857,0.142857,0,Lower West Side,41.889254,-87.675381
71,Old Town,0.000000,0.083333,0.000000,0.000000,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.000000,0.000000,0.083333,0.083333,0,Near North Side,41.905390,-87.641790
35,Humboldt Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0,Humboldt Park,41.899070,-87.719470
36,Illinois Medical District,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.142857,0.142857,0.000000,0.142857,0.142857,0.000000,0.000000,0.000000,0.000000,0.142857,0.142857,0,Near West Side,41.875570,-87.676520
91,Sheffield Neighbors,0.000000,0.083333,0.000000,0.000000,0.083333,0.083333,0.000000,0.083333,0.083333,0.083333,0.000000,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0,Lincoln Park,41.921691,-87.653488
38,Jackowo,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.142857,0.142857,0.000000,0.142857,0.142857,0.000000,0.000000,0.000000,0.000000,0.142857,0.142857,0,Avondale,41.884250,-87.632450
39,K-Town,0.000000,0.100000,0.000000,0.000000,0.100000,0.000000,0.000000,0.100000,0.100000,0.100000,0.000000,0.100000,0.100000,0.100000,0.100000,0.000000,0.100000,0,North Lawndale,41.921850,-87.697350
89,Saint Ben's,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.142857,0.142857,0.000000,0.142857,0.142857,0.000000,0.000000,0.000000,0.000000,0.142857,0.142857,0,North Center,41.884250,-87.632450
43,Lake View,0.076923,0.076923,0.076923,0.076923,0.076923,0.000000,0.000000,0.076923,0.076923,0.076923,0.000000,0.076923,0.076923,0.076923,0.076923,0.000000,0.076923,0,Lake View,41.939820,-87.656820


In [36]:

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [37]:


# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(chi_merged['Latitude'], chi_merged['Longitude'], chi_merged['Neighborhood'], chi_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters